#IPerl Notebook with Perl Data Language to process a simple thin layer chromatography experiment

Author: Demian Riccardi

Acknowledgements: This notebook was derived from suggestions and code posted to the PDL email list by Chris Marshall and Zaki Mugdahl

###How came about? 
A simple thin-layer chromatography experiment was carried out in an analytical chemistry lab at Earlham College, Spring 2015. The students had designed a protocol for measuring the arginine in dietary supplements. My goal is to develop a PDL script that will process the images and estimate sample content based on the images of known and unknown concentrations.  This notebook is intended to be a step in that direction. 

####TODO: after approach verified; automate analysis and spit out numbers. 

##### Note: there is a web application that one can load up an image and analyze spots, which may be useful for comparisons.

####Load the libraries, set up an IPerl helper function for displaying pdl images, load up the jpeg (already cropped), and display inline 

In [1]:
use Modern::Perl;
use PDL;
use PDL::IO::Pic;
use PDL::Image2D;

IPerl->helper( pdl_as_png => sub {
    my ($self, $pdl, @opts) = @_;
    my $tmp = File::Temp->new( SUFFIX => '.png' );
    $pdl->wim( $tmp->filename );
    my $png_displayable = IPerl->png( $tmp->filename, @opts );
    $png_displayable->iperl_data_representations; # make sure to build from filename
    $png_displayable;
});

#load
my $tlc = rpic('Desktop/TLC_arg.jpg');
say $tlc->info;
IPerl->pdl_as_png( $tlc, height => 100 );


PDL: Byte D [3,2625,116]


###Description of contents

There are 15 runs, one per lane.  The first 8 spots correspond to the standards (mg/ml of Arginine):

|Lanes    |   [Arg] (mg/ml)|
|:---------|:----------------
|1,2      |     0.4
|3,4      |     0.3
|5,6      |     0.2
|7,8      |     0.1
|9-11     |     unknown A
|12-15    |     unknown B


# 1. Clean-up the background
### First convert to greyscale by calculating the average pixel value 

In [2]:
my $tlc_g = $tlc->float->average;
say $tlc_g->info;
IPerl->pdl_as_png( $tlc_g, height => 100 );

PDL: Float D [2625,116]


### Select background pixels based on the white part of the image

pdl method used: 

        pct
                Return the specified percentile of all elements in a
                piddle. The specified percentile (p) must be between
                0.0 and 1.0. When the specified percentile falls
                between data points, the result is interpolated.

In [3]:
my $back_mask = ($tlc_g > $tlc_g->pct(0.1));
say $back_mask->info;
my $ones  = $back_mask->sum;
say $ones, ' ones in the background';
IPerl->pdl_as_png( $back_mask, height => 100 );

PDL: Float D [2625,116]
274025 ones in the background


### Average the background of the original RGB image using this mask

pdl method used

            where           Use a mask to select values from one or more data PDLs
            whereND         `where' with support for ND masks and threading

In [4]:
say $tlc->mv(0,-1)->info;
my $background = $tlc->mv(0,-1)->whereND($back_mask);
say $background->info;
my $b_rgb = $background->float->average;
say $b_rgb;
my $b_color = $b_rgb->norm;

PDL: Byte D [2625,116,3]
PDL: Byte D [274025,3]
[150.223 142.334 135.719]


[0.607015 0.575135 0.548408]


### Calculate the color direction for each pixel and the deviation from the background direction

In [5]:
my $tlc_unit = $tlc->float->norm; #each pixel
say $tlc_unit->info;
my $tlc_cos = $tlc_unit->inner($b_color);
IPerl->pdl_as_png( $tlc_cos, height => 100 );

PDL: Float D [3,2625,116]


### Switch to sin to make nice summable the spots

In [6]:
my $tlc_sin = sqrt(1-$tlc_cos);
$tlc_sin /= $tlc_sin->max;
IPerl->pdl_as_png( $tlc_sin, height => 100 );

###And now for a nasty extraction of spots, "by eye" using a square of same size.  Something fancier should be possible via PDL::Image2D

In [7]:
my $spot1 = $tlc_sin->slice("60:140","20:110");
say $spot1->sum;
IPerl->pdl_as_png( $spot1, height => 100 );

my $spot2 = $tlc_sin->slice("190:270","25:115");
IPerl->pdl_as_png( $spot2, height => 100 );

my $spot3 = $tlc_sin->slice("380:460","25:115");
IPerl->pdl_as_png( $spot3, height => 100 );

my $spot4 = $tlc_sin->slice("520:600","15:105");
IPerl->pdl_as_png( $spot4, height => 100 );
#print 'spot2 ', $spot2->sum;

my $spot5 = $tlc_sin->slice("720:800","0:90");
IPerl->pdl_as_png( $spot5, height => 100 );

my $spot6 = $tlc_sin->slice("860:950","0:90");
IPerl->pdl_as_png( $spot6, height => 100 );

my $spot7 = $tlc_sin->slice("1040:1120","0:90");
IPerl->pdl_as_png( $spot7, height => 100 );

my $spot8 = $tlc_sin->slice("1170:1250","0:90");
IPerl->pdl_as_png( $spot8, height => 100 );

my $spot9 = $tlc_sin->slice("1490:1570","0:90");
IPerl->pdl_as_png( $spot9, height => 100 );

my $spot10 = $tlc_sin->slice("1660:1740","0:90");
IPerl->pdl_as_png( $spot10, height => 100 );

my $spot11 = $tlc_sin->slice("1860:1940","0:90");
IPerl->pdl_as_png( $spot11, height => 100 );

my $spot12 = $tlc_sin->slice("2070:2150","0:90");
IPerl->pdl_as_png( $spot12, height => 100 );

my $spot13 = $tlc_sin->slice("2210:2290","0:90");
IPerl->pdl_as_png( $spot13, height => 100 );

my $spot14 = $tlc_sin->slice("2370:2450","0:90");
IPerl->pdl_as_png( $spot14, height => 100 );

my $spot15 = $tlc_sin->slice("2510:2590","0:90");
say "you are seeing spot15";
IPerl->pdl_as_png( $spot15, height => 100 );

2604.63330078125
you are seeing spot15


In [8]:
print 'spot1 ', $spot1->sum . "\n";
print 'spot2 ', $spot2->sum . "\n";
print 'spot3 ', $spot3->sum . "\n";
print 'spot4 ', $spot4->sum . "\n";
print 'spot5 ', $spot5->sum . "\n";
print 'spot6 ', $spot6->sum . "\n";
print 'spot7 ', $spot7->sum . "\n";
print 'spot8 ', $spot8->sum . "\n";
print 'spot9 ', $spot9->sum . "\n";
print 'spot10 ', $spot10->sum . "\n";
print 'spot11 ', $spot11->sum . "\n";
print 'spot12 ', $spot12->sum . "\n";
print 'spot13 ', $spot13->sum . "\n";
print 'spot14 ', $spot14->sum . "\n";
print 'spot15 ', $spot15->sum . "\n";

my $a = ($spot1->sum+$spot2->sum)/2;
my $b = ($spot3->sum+$spot4->sum)/2;
my $c = ($spot5->sum+$spot6->sum)/2;
my $d = ($spot7->sum+$spot8->sum)/2;

say foreach reverse($a,$b,$c,$d);


spot1 2604.63330078125
spot2 2333.97314453125
spot3 1845.08703613281
spot4 1363.78955078125
spot5 368.448944091797
spot6 466.89697265625
spot7 390.204010009766
spot8 165.708572387695
spot9 1493.82019042969
spot10 1269.26599121094
spot11 1087.25646972656
spot12 375.565093994141
spot13 335.383422851562
spot14 444.922973632812
spot15 516.956787109375
277.95629119873
417.672958374023
1604.43829345703
2469.30322265625
